# Builder123D Sandbox

## Important Links:
* [Builder123D Github](https://github.com/gumyr/build123d)
    * [Documentation](https://build123d.readthedocs.io/en/latest)
    * [Geometric & Topological Primitives](https://build123d.readthedocs.io/en/latest/direct_api_reference.html#)
    * [1D/2D/3D Objects](https://build123d.readthedocs.io/en/latest/objects.html#)
    * [Selectors](https://build123d.readthedocs.io/en/latest/key_concepts.html#selectors)
* [VSCode Visualization Plugin](https://marketplace.visualstudio.com/items?itemName=bernhard-42.ocp-cad-viewer)
    * [Source](https://github.com/bernhard-42/vscode-ocp-cad-viewer/tree/main)
    * [3CAD Viewer](https://github.com/bernhard-42/three-cad-viewer?tab=readme-ov-file), base for the plugin.
    * [Animation Primer](https://github.com/bernhard-42/bd_animation)
* [Simple CAM package](https://ocp-freecad-cam.readthedocs.io/en/latest/index.html)

## Other Links
* [List of Links](https://github.com/phillipthelen/awesome-build123d)
* [Parts Library](https://github.com/gumyr/bd_warehouse)
* [Gridfiniy](https://github.com/Ruudjhuu/gridfinity_build123d)
* [Alternate Viewer](https://github.com/yeicor-3d/yet-another-cad-viewer)



In [1]:
from build123d import *
from builder123d_utils import *
from ocp_vscode import show, show_object, show_clear
from pathlib import Path
from operations import OperationFace, stock_make, OperationBore
from tools import Tool
import numpy as np
from typing import List, Tuple


In [16]:
# Center includes Z is centered.
# Pitch sign relative to height sign controls direction of rotation.
# If signs are the same, rotation is CCW
# Negative normal flips the start point across the Y-axis
e = Edge.make_helix(-1,0,5, center=(5,5,-1))
e.location
show_clear()
show(Wire(e))

ZeroDivisionError: float division by zero

In [3]:

# Alsways need 3 pieces of geometry for a machining operation:
# 1) The part
# 2) The stock
# 3) The tool

# 1) Load a 3D model from a file
fn_part = 'simple-part-with-holes.step'
part = import_step(fn_part)
part.label = Path(fn_part).stem
part.color = Color("Orange")
part.color = Color("Gray",alpha=0.5)
show(part)

# 2) Generate fitting stock
stock = stock_make(part, margin=2)

# 3) Create tool and home it
tool = Tool(diameter=3.175, length=25.4)
#tool.to_stock_home(stock)

Using port 3939 taken from config file
Jupyter console not installed
+

In [4]:
# Status:
# Bore operation implemented, untested.
# Curved wires not implemented yet. - operations.py:445
# Curved lofts for generating swept volume not implemented yet

op = OperationBore(part, tool, stock)
bores = op.bores

# [b.show() for b in bores]
